In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

pinecone_key = os.getenv("PINECONE_API_KEY")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")


In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec

index_name = "hybrid-search-langchain-pinecone"

# Init Pinecone client
pc = Pinecone(api_key=pinecone_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension=384, # dimension of dense vector, sentence transformer hugging face
        metric = 'dotproduct', # sparse value supoorts dot product
        spec = ServerlessSpec(cloud = 'aws', region = 'us-east-1')
    )
index = pc.Index(index_name)

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")


In [14]:
from pinecone_text.sparse import BM25Encoder


bm25_encoder = BM25Encoder().default()

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/river/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [10]:
sentences = ["In 2021, I visisted Switzerland",
             "In 2022, I visited Paris",
             "In 2023, I visited London"]

# tfidf values on the sentences
bm25_encoder.fit(sentences)

# store values to json file
bm25_encoder.dump("bm25_values.json")

#load to your bm25 object
bm25_encoder = BM25Encoder().load("bm25_values.json")

NameError: name 'bm25_encoder' is not defined

In [ ]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,
                                          sparse_encoder = bm25_encoder,
                                          index=index)

In [ ]:
retriever.add_texts(sentences)
retriever.invoke("WHich was the last visited city")